In [ ]:
# Install required packages
!pip install torch torchvision scikit-learn facenet-pytorch pillow numpy


In [ ]:
import os
import torch
import pickle
import numpy as np
from sklearn.svm import SVC
from facenet_pytorch import InceptionResnetV1, MTCNN
from torchvision import transforms
from PIL import Image

# Load MTCNN for face detection
mtcnn = MTCNN()

# Load FaceNet model for embeddings
facenet = InceptionResnetV1(pretrained='vggface2').eval()

# Allowed image extensions
VALID_EXTENSIONS = {'.jpg', '.jpeg', '.png'}

def load_image(img_path):
    transform = transforms.Compose([
        transforms.Resize((160, 160)),
        transforms.ToTensor()
    ])
    img = Image.open(img_path).convert("RGB")
    return transform(img).unsqueeze(0)

def extract_embeddings(data_path):
    labels = []
    embeddings = []
    label_map = {}
    label_counter = 0
    
    for person_name in os.listdir(data_path):
        person_path = os.path.join(data_path, person_name)
        if not os.path.isdir(person_path):
            continue
        
        if person_name not in label_map:
            label_map[person_name] = label_counter
            label_counter += 1
        
        for image_name in os.listdir(person_path):
            image_path = os.path.join(person_path, image_name)
            _, ext = os.path.splitext(image_name)
            if ext.lower() not in VALID_EXTENSIONS:
                continue  # Skip non-image files
            
            img = load_image(image_path)
            
            with torch.no_grad():
                embedding = facenet(img).numpy().flatten()
                embeddings.append(embedding)
                labels.append(label_map[person_name])
    
    return np.array(embeddings), np.array(labels), label_map

# Load dataset and extract embeddings
data_path = "E:\SWE_ACADEMICS\3-1\ML Lab\Project\ML_final\Dataset"  # Update this path accordigly
embeddings, labels, label_map = extract_embeddings(data_path)

# Train SVM classifier
svm_classifier = SVC(kernel='linear', probability=True)
svm_classifier.fit(embeddings, labels)

# Save the trained model
model_data = {"model": svm_classifier, "label_map": {v: k for k, v in label_map.items()}}
with open("face_classifier.pkl", "wb") as f:
    pickle.dump(model_data, f)

print("Training complete. Model saved as face_classifier.pkl.")